In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/questionanswer-dataset/text_data_toc.csv
/kaggle/input/questionanswer-dataset/S10_question_answer_pairs.txt
/kaggle/input/questionanswer-dataset/S09_question_answer_pairs.txt
/kaggle/input/questionanswer-dataset/S08_question_answer_pairs.txt
/kaggle/input/questionanswer-dataset/text_data/S08_set1_a4.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S08_set1_topics.txt
/kaggle/input/questionanswer-dataset/text_data/S10_set5_a6.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S09_set5_a5.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S09_set2_a10.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S10_set3_a5.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S09_set3_a1.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S09_set5_a2.txt.clean
/kaggle/input/questionanswer-dataset/text_data/S08_set2_topics.txt
/kaggle/input/questionanswer-dataset/text_data/S09_set2_topics.txt
/kaggle/input/questionanswer-dataset/text_data/S09_set2_a1.

In [2]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
#!pip install -U transformers
!pip install plotly
!pip install accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

2024-07-13 20:20:50.411478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 20:20:50.411629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 20:20:50.552700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Key Configuration Parameters of Gemma models

In [4]:
class CFG:
  # Set a seed value for reproducibility (e.g., same random number generation)
  seed = 123
  # Path to the directory containing your dataset
  dataset_path = "/kaggle/input/kaggle-docs/questions_answers"
  # Name of the pre-trained Gemma model to use ("gemma_2b_en" for English)
  preset = "gemma_2b_en"
  # Maximum length of an input sequence for training (in tokens)
  sequence_length = 512
  # Number of samples processed in a single training step (adjust based on hardware)
  batch_size = 1
  # Number of times to iterate through the entire dataset during training
  epochs = 10
  # Optimizer used for training the model (Adam with a learning rate of 3e-5)
  optimizer = keras.optimizers.Adam(learning_rate=3e-5)
  # Loss function used during training (Sparse Categorical Crossentropy)
  loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [5]:
import gc
gc.collect()

# Tokenizer
tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma_instruct_2b_en")
# Model Loading
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
#gemma_llm.summary()

Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'mod

In [ ]:
# Define a template string with placeholders for 'instruction' and 'response' to structure the output.
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Use the `format` method to substitute the placeholders in the template string with actual values.
# Here, 'instruction' is set to a question about Kaggle competitions, and 'response' is initially empty.
prompt = template.format(
    instruction="How do Kaggle competitions work?",
    response="",
)

display(Markdown(gemma_lm.generate("How do Kaggle competitions work?", max_length=256)))

## Fine-Tuning with LoRA (Low-Rank Adaptation)

In [ ]:
gemma_lm.summary()

In [ ]:
!pip install peft

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=6, #"rank" of the LoRA adapters. This controls the size and computational efficiency of the adapters.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

#### change rank in aobve and check the difference

### Applying LoRA to Gemma

In [ ]:
gemma_lm.backbone.enable_lora(rank=6)
gemma_lm.summary()

## Data Preparation

In [ ]:
train_df = pd.read_csv("/kaggle/input/kaggle-docs/questions_answers/data.csv")
train_df.head(5)

In [ ]:
train_df.tail(5)

In [ ]:
train_df.shape

## Preprocess the data

In [15]:
def preprocess(df):
    df['prompt'] = df[['Category', 'Question', 'Answer']].apply(
        lambda row: f"\n\nCategory: {row['Category']}\n\nQuestion:\n{row['Question']}\n\nAnswer:\n{row['Answer']}", axis=1
    )
    return df['prompt'].tolist()

data = preprocess(train_df.copy()) 

In [16]:
data[59]

'\n\nCategory: dataset\n\nQuestion:\nWhat are the technical specifications of Kaggle Datasets?\n\nAnswer:\nKaggle Datasets allows you to publish and share datasets privately or publicly. We provide resources for storing and processing datasets, but there are certain technical specifications:\n\n* 100GB per dataset limit\n* 100GB max private datasets (if you exceed this, either make your datasets public or delete unused datasets)\n* A max of 50 top-level files (if you have more, use a directory structure and upload an archive)\n\nWhen you upload a dataset we apply certain processing steps to make the dataset more usable.\n\n* A complete archive is created so the dataset can be easily downloaded later\n* Any archives (e.g., ZIP files) that you upload are uncompressed so that the files are easily accessible in Notebooks (directory structure is preserved)\n* Data types for tabular data files are automatically detected (e.g., geospatial types)\n* Column-level metrics are calculated for tabu

## Fine-tuning with LoRA (Low-Rank Adaptation)

In [17]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

60/60 ━━━━━━━━━━━━━━━━━━━━ 1975s 32s/step - loss: 3.0819 - sparse_categorical_accuracy: 0.4644


In [ ]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
prompt = template.format(
    instruction="How do Kaggle competitions work?",
    response="",
)
display(Markdown(gemma_lm.generate("How do Kaggle competitions work?", max_length=256)))

## Inference after fine-tuning

#### Overall, the results are acceptable after fine-tuning but remain incomplete. To address this, a hybrid system incorporating both fine-tuning and RAG could significantly improve the model.

## Retrieval-Augmented Generation (RAG)

## Building a Retrieval-Augmented Generation (RAG) Model with Gemma and Langchain

In [18]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1
!pip3 install langchain sentence-transformers chromadb langchainhub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.3.1 requires fsspec==2024.3.1, but you have fsspec 2023.10.0 which is incompatible.
s3fs 2024.3.1 requires fsspec==2024.3.1, but you have fsspec 2023.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions 

In [19]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = UserSecretsClient().get_secret("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token 

# Login to Hugging Face
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#### 2. We import the necessary modules for Langchain integration and set up a Hugging Face Endpoint

In [20]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM

#### Overall, this code snippet suggests you're setting up a Langchain pipeline to leverage a Hugging Face model for text generation tasks.

In [23]:
## Define a custom LangChain
from typing import Any, Optional, List, Mapping
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

In [24]:
class GemmaLLM(LLM):
    """
    Custom LangChain LLM class for Keras Gemma model.
    """

    # Class attributes to store the model and desired generation length
    model: Any = None
    n: int = None

    def __init__(self, keras_model, n):
        """
        Constructor for the GemmaLMM class.
        Args:
            keras_model: The loaded Keras Gemma model.
            n: The desired maximum generation length (number of words).
        """
        super(GemmaLLM, self).__init__()  # Call parent class initialization
        self.model = keras_model
        self.n = n

    @property
    def _llm_type(self) -> str:
        """
        Property to identify the LLM type as "Gemma".
        """
        return "Gemma"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs) -> str:
        """
        Core method for generating text using the GemmaLLM.

        Args:
            prompt: The text prompt or query for generation.
            stop: Optional list of stop words or phrases (not used here).
            **kwargs: Additional keyword arguments for generation (not used here).

        Returns:
            The generated text as a string.
        """

        generated = self.model.generate(prompt, max_length=self.n)

        # Post-processing to extract summarized text (if applicable)
        split_string = generated.split("SUMMARY:", 1)
        if len(split_string) > 1:
            return split_string[1].lstrip('\n')
        else:
            return generated.lstrip('\n')

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """
        Property to return identifying parameters of the LLM (n in this case).
        """
        return {"n": self.n}

In [25]:
llm_model = GemmaLLM(gemma_lm, 256)

#### Overall, this custom GemmaLLM class provides a wrapper for the Keras Gemma model, making it compatible with LangChain's LLM framework for text generation tasks.

#### 3. Chose a dataset

In [26]:
# Load a document
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Kaggle")
data = loader.load()

In [27]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

In [28]:
# Initialize a CharacterTextSplitter to split text into chunks of 1000 characters without overlap
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Assuming 'data' contains the text to be split, split it into manageable chunks using the text splitter
docs = text_splitter.split_documents(data)

In [35]:
pip install langchain-huggingface

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 64.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_huggingface import HuggingFaceEmbeddings

ModuleNotFoundError: No module named 'langchain_huggingface'

In [30]:
import sentence_transformers

/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py:533: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  import torch._dynamo as dynamo  # noqa: F401


In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [31]:
# Initialize SentenceTransformerEmbeddings with a specific model to create embeddings for text chunks
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a Chroma database to store embeddings of the text chunks, using the specified embedding function
db = Chroma.from_documents(docs, embedding_function)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1535, in _get_module
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.cohere.configuration_cohere'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_33/2194975298.py", line 2, in <module>
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM

In [40]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

In [ ]:
# Configure the retriever using the Chroma vector store `db`, setting the search type to 'mmr' and defining search parameters
retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})

# Pull a RAG prompt configuration from the Langchain hub
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the retrieved documents, concatenating them with double newlines
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create a RAG chain by composing different components:
# `retriever | format_docs` creates the context by fetching and formatting documents
# `RunnablePassthrough()` passes the question directly as is
# `prompt` applies the RAG prompt configuration
# `llm` represents the language model to generate answers (not defined in this snippet)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_model
)

In [ ]:
rag_chain.invoke("Who own Kaggle?")

In [ ]:
# Import the ConversationBufferMemory class from the langchain.memory module.
from langchain.memory import ConversationBufferMemory

# Import the ConversationalRetrievalChain class from the langchain.chains module.
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# Initialize a ConversationBufferMemory object with a specific key for storing chat history.
# The 'return_messages' flag is set to True to ensure that the stored messages are retrievable.
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Define a string that specifies how the prompt should be structured for rephrasing follow-up questions.
# This template includes placeholders for chat history and the follow-up question that will be rephrased.
custom_template = """Given the following conversation and a follow-up question, rephrase the follow-up question to be a standalone question, in its original English.
                        Chat History:
                        {chat_history}
                        Follow-Up Input: {question}
                        Standalone question:"""

# Create a PromptTemplate object using the custom template string.
# This object will be used to format the input for the language model.
from langchain.prompts import PromptTemplate  # Importing PromptTemplate, assuming it's necessary and was omitted in the original code
CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

# Instantiate a ConversationalRetrievalChain object by specifying the language model (llm),
# the type of chain, a retriever component (assumed to be part of a database 'db'), the memory component,
# and a custom prompt template for question condensation.
# This creates a chain that can handle conversational context and retrieval-based tasks.
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_model,  
    chain_type="stuff",  # 'stuff' should be replaced with the actual type of chain intended to use
    retriever=db.as_retriever(),  # Assuming 'db' is a predefined database object with a method to create a retriever
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT
)

## Prompt Engineering

In [ ]:
prompt = template.format(
    instruction="""
**Act as an expert Kaggle assistant.**

* **Focus on Actionable Insights:**  Provide direct answers, links to Kaggle resources, or suggest search terms that will immediately help me with my Kaggle tasks.
* **Understand My Intent:** If a question is ambiguous, ask for clarifications or provide examples to help me rephrase it effectively. 
* **Be Proactive:**  Offer guidance even if I don't have a precise question. Suggest popular datasets, trending competitions, or widely-used techniques based on my interests.  
* **Stay Concise:**  Deliver the most important information first.
**Let's begin! What's your Kaggle-related question or task?** 
""" ,
    response="",
)

In [ ]:
display(Markdown(gemma_lm.generate(prompt, max_length=256)))

In [ ]:
display(Markdown(gemma_lm.generate("Introduce me to the Kaggle platform.", max_length=256)))